In [1]:
import pandas as pd

In [240]:
def to_list(sample):

    import ast

    x = ast.literal_eval(sample)
    x = [n.strip() for n in x]
#     print(len(x))
    
    return x


def test_split(sample): 
    import re
    pp = re.compile('(\[?\s?\[\[.*(BODY\s\d).*\]\]\s?\,?)')
    print(split_sent(sample)[0])
    print()
    print(split_sent(sample.replace('\'',' '))[0])
    print()
    print(pp.sub('', split_sent(sample.replace('\'',' '))[0]))

In [807]:
# split_sent function
import re
from tqdm.auto import tqdm

tqdm.pandas()


def make_new_dataframe(dataset):
    sample = dataset
    
    # 문장 단위로 전처리 : split_sent function
    splited = sample.progress_apply(lambda row: split_sent(' '.join(row['paragraphs'])), axis=1)

    # 문장에 해당되는 index, title, section을 부여
    # 문장 개수 획득
    lengths = []
    
    for i in range(len(splited)):
        lengths.append(len(splited.iloc[i]))
    
    # 문장 개수만큼 index, title, section copy
    book_ids   = []
    body_nums  = []

    for i in range(len(sample)):
        book_ids.append([sample.iloc[i]['book_id']]*lengths[i])
        body_nums.append([sample.iloc[i]['body_num']]*lengths[i])
    
    
    temp_df_list = []

    for i in range(len(splited)):
        for j in range(len(splited.iloc[i])):
            temp_df_list.append([book_ids[i][j], body_nums[i][j], splited.iloc[i][j]])


    df = pd.DataFrame(temp_df_list)
    df.columns=['book_id', 'section', 'sentences']
    
    return df
  



# def make_new_dataframe_old(dataset):
#     new_df = pd.DataFrame()
    
#     for i in tqdm(range(len(dataset))):
#         new_df = pd.concat([new_df, func(dataset.iloc[i])])
    
#     new_df = new_df.reset_index()
#     new_df.columns = ['sent_index', 'parse_index', 'title', 'section', 'sentence']
#     return new_df


# def func(df):
#     data = df
#     splited_list = split_sent(str(data['text']))
#     list_len = len(splited_list)
    
#     indices = [data['index']] * list_len
#     titles = [data['title']] * list_len 
#     sections = [data['section']] * list_len
    
#     df = pd.DataFrame(zip(indices, titles, sections, splited_list))
    
#     return df 


# 수정 후 

def split_sent(text:str)->list:
    # split 시 무시할 패턴 정의  
    # 숫자. 로 끝나는 경우 
    check = """(eqs?\.)|(figs?\.)|(sect\.)|\     # eq.(eqs.), fig.(figs.), sec.(SEC.),
    (sec\.)|(et\.)|(\s?al\.)|(\s?et\.?\sal\.)|\  #  et. al. 
    (S?\.?\s?\d*ff\.)$|(dvbl\.)|(vgl\.)|\        # 추가: '.857ff.','3) Vgl.'  11.18
    ^(e\.)|(\s?\(?[A-Z]{1,4}\.)|^([A-Z]\.)|\     # 시작 시 e. g. (S. S. 로 시작할 경우, ABCD. 와 같은 경우 추가(+11.18)
    ([가-힣]{1,2}\s\.)$|\                         # 다. 일경우만 제외하는 등 변경 필요
    (\(?e\.\s?g\.)|(\(?i\.\s?e\.)|\
    ([MDCLXVI]{1,2}\.)$\                         # 로마자 추가
    |(dix.)\
    |(no\.)|([0-9]\.)$|(vs\.)|(ref\.)|(cf\.)\    # no. 1. vs. ref.(Ref.) cf.(Cf.)
    """
    pp = re.compile(check, re.VERBOSE|re.IGNORECASE)
    na_count = 0

        
    # 예외처리: 문장 끝에 .(공백없음)으로 끝나거나 .이 없는 경우
    sample = new_sample(text)
    splited_list = []
    prev_end = 0
    
    try:
        # .으로 끝나는 모든 경우를 find
        for sent in re.finditer('(\.\s)', sample):
            start = sent.span()[0]
            end = sent.span()[1]

            if start < 6:
                token = sample[start-start:end-1]
            else:
                token = sample[start-6:end-1]

    #         print()
    #         print(pp.search(token))
    #         print(token)
    #         print(prev_end, end)
    #         print(len(sample))

            # '. ' 앞뒤에 check에 해당하는 패턴이 없는 경우 문장으로 나누기
            if (pp.search(token) == None):
                if re.search('(\s\(?\w\.\)?)$', token) == None:  # 위의 check에서 제대로 동작 안 해서 따로 집어 넣음
                    splited_list.append(sample[prev_end:end-1])
                    prev_end = end

            elif pp.search(token) != None:
                if len(sample) == end:
                    splited_list.append(sample[prev_end:end-1])
                    
    except:
        print(f'error occured!')
        print(sample)
    return splited_list



def new_sample(sample):
    try:
        if sample[-2:] == '. ':
            new_sample = sample

        elif sample[-1] == '.':
            new_sample = sample+' '

        elif sample[-1] != '.':
            new_sample = sample+'. '

        else:
            print('please check the data!')
    except:
        new_sample = sample
        
    return new_sample


# data check
def check_data(dataframe, freq:int = 100) -> None:
    for i in range(len(dataframe)):
        if i % freq == 0:
            print(f'index {i}th: ')
            print(dataframe.loc[i]['sentence'])
            print()
            
            

In [465]:
def reconstruct_para(book):

    body = []
    contents = []
    prev_content_tag = ''

    for i in range(len(book)):
        content_tag = book.iloc[i].tag
        content = book.iloc[i].paragraphs

        if content_tag != prev_content_tag:     
            contents.append(body)
            body = []
            body.extend(content)

        else:
            body.extend(content)

            if i == len(book)-1:
                contents.append(body)

        prev_content_tag = content_tag


    contents = contents[1:]
    
    return contents


def make_book_df(book_id, contents):
    book_id = book_id
    book_df = pd.DataFrame(zip(contents))
    book_df = book_df.reset_index()
    book_df.loc[:,'index'] = book_df['index'].apply(lambda idx: idx+1)
    book_df.columns = ['body_num', 'paragraphs']
    book_df['book_id'] = book_id
    book_df = book_df[['book_id', 'body_num', 'paragraphs']]   
    return book_df


def reconstruct_df(df):

    from tqdm import tqdm 

    new_df = pd.DataFrame()  
    book_list = df.book.unique()
    
    for book_id in tqdm(book_list):
        book = df[df.book == book_id]
        contents = reconstruct_para(book)
        book_df = make_book_df(book_id, contents)
        new_df = pd.concat([new_df, book_df])

    return new_df


In [854]:
book_list = pd.read_excel("/Users/siri/Documents/Project/Corpus/master progress.xlsx")
book_list['ID'] = book_list['BOOK ID'].apply(lambda row: row[2:])
book_list = book_list[['ID', 'BOOK ID', '도서제목']]

In [855]:
book_list

,ID,BOOK ID,도서제목
0,1101,LP1101,미디어를 통해서 배우는 법학 기초
1,1092,LP1092,금전거래와 법: 거래를 통해 배우는 민법 1
2,1113,LP1113,부동산거래와 법
3,1002,LP1002,경영판단원칙과 형법-체계간 원칙(intersystemic principle)으로서 ...
4,1124,LP1124,소비자와 권리
...,...,...,...
429,0766,LP0766,민사판례연구(5) (제 2 판)
430,0767,LP0767,민사판례연구(6) (제 2 판)
431,0768,LP0768,민사판례연구(7) (제 2 판)
432,0769,LP0769,민사판례연구(8)


In [856]:
_ = list(df['book'].unique())

In [857]:
cond = [b in _ for b in book_list['BOOK ID'] ]

In [877]:
cond.count(False)

389

In [862]:
cond

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 

In [880]:
book_list_tmp = book_list[cond].reset_index(drop=False).drop('index', axis=1)

In [896]:
cond = [ b in list(book_list_tmp['BOOK ID'][:].values) for b in _ ]

In [899]:
for b in _:
    if b == 

['ME0053',
 'ME0202',
 'LP1096',
 'LP1140',
 'LB0067',
 'LB0076',
 'LB0080',
 'LB0083',
 'LB0091',
 'LB0097',
 'LB0102',
 'LB0161',
 'LB0282',
 'LB0290',
 'LB0350',
 'LB0357',
 'LB0362',
 'LP0036',
 'LP0266',
 'LP0982',
 'LP1001',
 'LP1156',
 'MK0208',
 'LB0107',
 'LB0122',
 'LB0167',
 'LB0236',
 'LB0241',
 'LB0253',
 'LB0305',
 'LB0363',
 'LP1036',
 'LP1108',
 'ME0213',
 'ME0231',
 'MK0095',
 'ME0221',
 'ME0224',
 'MK0031',
 'MK0179',
 'MK0205',
 'LP0231',
 'ME0176',
 'ME0262',
 'MK0035',
 'MK0094',
 'MK0123',
 'MK0130',
 'MK0150',
 'MK0152',
 'MK0225']

In [ ]:
# unicode 제거 필요 \U0010beb7
df = pd.read_excel('./data_splited_para.xlsx')

In [292]:
df['tag'] = df.paragraphs.apply(lambda line: to_list(line)[0])
df['paragraphs'] = df.paragraphs.apply(lambda line: to_list(line)[1:])
df = df.drop(['Unnamed: 0', 'date', 'split_file'], axis=1)
df = df[['book', 'tag', 'paragraphs']]
df

,book,tag,paragraphs
0,ME0053,[[BODY 1]],"[서문 신경계 질환에 대한 물리치료 중재는 임상에서 중요한 영역을 차지하고 있으며,..."
1,ME0053,[[BODY 2]],"[01 임상신경학 Clinical neurology, Keyword, 신경세포 (n..."
2,ME0053,[[BODY 2]],"[대뇌의 엽 대뇌겉질의 주요엽은 이마엽(frontal lobe), 마루엽(parie..."
3,ME0053,[[BODY 2]],[바닥핵 손상 바닥핵의 일부 또는 전부가 손상되면 경축(rigidity)과 비정상적...
4,ME0053,[[BODY 2]],[그물구성체 활성계의 활성은 각성을 촉진하는 한편 그물구성체의 활성계의 억제는 수면...
...,...,...,...
18666,MK0225,[[BODY 7]],[마지막으로 연구와 관계된 문헌 고찰을 간략 히 검토하여 포함시킨다. 이 부분은 연...
18667,MK0225,[[BODY 7]],[(5) 기관 검토 어떤 종류의 연구이든 IRB나 기타 유사한 기관의 승인을 얻는 ...
18668,MK0225,[[BODY 7]],[연구 진행에 동료나 공동 연구자의 도움이 필요할 수 있다. (2) 연구 진행에 필...
18669,MK0225,[[BODY 7]],[때로 는 이러한 기록이 의무 기록의 한 부분이 아니어서 의무 기록을 관리하는 부서...


In [469]:
new_df = reconstruct_df(df)
new_df

,book_id,body_num,paragraphs
0,ME0053,1,"[서문 신경계 질환에 대한 물리치료 중재는 임상에서 중요한 영역을 차지하고 있으며,..."
1,ME0053,2,"[01 임상신경학 Clinical neurology, Keyword, 신경세포 (n..."
2,ME0053,3,[02 신경계 물리치료의 원리 Principle of neurological phy...
3,ME0053,4,"[03 뇌졸중 Stroke Keyword, 뇌졸중 (cerebral vascular..."
4,ME0053,5,"[04 파킨슨병 Parkinson’s diseases Keyword, 바닥핵 (ba..."
...,...,...,...
2,MK0225,3,[Chapter 3 재난과 질병 테러 및 바이러스 질환 1. 위험물질(hazardo...
3,MK0225,4,[Chapter 4 재난대책 한국의 재난관리대책 1. 재난관리체계의 역사적 전개와 ...
4,MK0225,5,[Chapter 5 재난응급의료체계 1. 병원 재난 계획의 원칙 1) 서론 이 장에...
5,MK0225,6,[Chapter 6 해외 긴급구조 의료장비 및 운영방안 서론 1. 개요 1) 재난 ...


In [809]:
sentence_df = make_new_dataframe(new_df)

  0%|          | 0/1108 [00:00<?, ?it/s]

In [822]:
sentence_df

,book_id,section,sentences
0,ME0053,1,"서문 신경계 질환에 대한 물리치료 중재는 임상에서 중요한 영역을 차지하고 있으며, ..."
1,ME0053,1,최근의 물리치료사 국가고시 체계는 임상 실무 중심의 교육과정 이수를 강조하고 있으며...
2,ME0053,1,이 책의 구성은 한국보건의료인 국가시험원 물리치료사 국가고시를 준비하기 위해 반드시...
3,ME0053,1,"이 책의 구성은 먼저 신경계의 구조와 기능, 각 구조의 손상에 따른 임상 양상을 이..."
4,ME0053,1,또 신경계 질환 중재의 기본적인 생리학적 기전과 치료적 원리를 설명하여 전반적으로 ...
...,...,...,...
365612,MK0225,7,성 헬렌 산의 화산 폭발에 대한 연구에서는 공동 연구자 그룹을 구성하여 장기간 화산...
365613,MK0225,7,"재난 연구에서 공동 연구는 재난을 완벽하게 이해하는 데 도움을 주고, 동시에 여러 ..."
365614,MK0225,7,7) 결론 재난 연구의 목적 중 하나는 임상 의사를 재난 현 장에 보내 재난으로 인...
365615,MK0225,7,"수색 과 구조, 응급의료 서비스의 대응, 응급 의학과의 병원 중재, 육체적 및 정신..."


In [823]:
sentence_df.to_csv('./sentence_df.tsv', sep='\t')

In [540]:
new_df[new_df['book_id'] == 'ME0053'].iloc[5].paragraphs

['05 치매 Dementia Keyword',
 '치매 (dementia) 인지 (cognition) 알츠하이머병 (Alzheimer’s disease) 간이 정신상태 검사 (Mini-Mental State Examination, MMSE) 기억력 장애 (memory impairment) 혈관성 치매 (vascular dementia) 아밀로이드 반 (amyloid plaque)',
 '학습 목표',
 '1. 치매의 정의에 대해 설명할 수 있다. 2. 치매의 발생빈도에 대해 설명할 수 있다. 3. 치매의 원인 및 병인에 대해 설명할 수 있다. 4. 치매의 분류 및 구분에 대해 설명할 수 있다. 5. 치매의 특징적인 임상양상을 설명할 수 있다. 6. 치매 환자에 대해 진단평가를 적용할 수 있다. 7. 치매 환자에 대해 물리치료를 적용할 수 있다. 8. 치매 환자에 대해 의학적 처치 및 관리를 적용할 수 있다. 노인은 노화에 따른 신경계의 변화로 뇌겉질 신경세포의 감소와 뇌 위축으로 인해 뇌 크기가 감소된다. 신체 건강에 있어 신경계와 관련된 정신질환의 문제는 중요한 변수가 되며, 특히 노인의 신체적, 정신적, 인지적 요소의 변화는 상호관련성을 가지고 있다. 치매는 암, 후천성 면역결핍증과 함께 세계보건기구가 정한 21세기 3대 질환의 하나로, 인간의 독립적인 생활 능력을 현저히 감소시키는 대표적인 만성 진행성 퇴행 뇌 정신 질환이다. 원인에 관계없이 기억력 장애와 인지기능의 장애로 인해 일상생활활동에 심각한 장애가 초래되는 경우는 치매로 진단할 수 있으며, 정상적인 노화과정에서 동반되는 기억력 및 정신 기능의 저하는 확실히 구별되는 질병이다. 또한 치매 발생의 증상, 종류, 발현 시기가 개인에 따라 매우 다양하며 발병 후 수년이 지나서야 증상을 느끼게 되는 경우도 있지만, 대부분이 진행성 장애이므로 결국 시간이 경과하면 심각한 일상생활능력이 감소하게 된다. 노령화에 따른 신경계의 변화는 다양한 문제를 야기하고 그중 일반적인 신경정신성 질환 문제 중 치매에 대해

In [485]:
new_df.to_csv('1-cycle_df.tsv', sep='\t')

In [23]:
med_book = []
law_book = []

for book in list(df.book.unique()):
    if 'M' in book :
        med_book.append(book)
    else:
        law_book.append(book)

In [28]:
print("original target med(34), law(20) total 54")
print(f"our dataset     med({len(med_book)}), law({len(law_book)}) total 51")

original target med(34), law(20) total 54
our dataset     med(20), law(31) total 51


In [54]:
book_cnt = df.book.value_counts().sort_index()
book_cnt

LB0067     127
LB0076     238
LB0080     814
LB0083     379
LB0091     131
LB0097     445
LB0102     510
LB0107      94
LB0122    1565
LB0161     208
LB0167     224
LB0236     211
LB0241     643
LB0253     641
LB0282     896
LB0290     692
LB0305     625
LB0350     633
LB0357     623
LB0362    1230
LB0363     256
LP0036     319
LP0231     105
LP0266     495
LP0982    1152
LP1001     245
LP1036     509
LP1096     121
LP1108     171
LP1140     170
LP1156     512
ME0053     235
ME0176     101
ME0202     125
ME0213      70
ME0221      14
ME0224      21
ME0231      35
ME0262     116
MK0031      75
MK0035     187
MK0094     164
MK0095      70
MK0123     407
MK0130     304
MK0150     859
MK0152     438
MK0179      79
MK0205     127
MK0208      80
MK0225     180
Name: book, dtype: int64

In [66]:
med_book_cnt = book_cnt.loc[med_book].sort_values(ascending=False)
med_book_cnt.describe()

count     20.000000
mean     184.350000
std      197.739934
min       14.000000
25%       73.750000
50%      120.500000
75%      199.000000
max      859.000000
Name: book, dtype: float64

In [67]:
law_book_cnt = book_cnt.loc[law_book].sort_values(ascending=False)
law_book_cnt.describe()

count      31.000000
mean      483.354839
std       361.043169
min        94.000000
25%       209.500000
50%       445.000000
75%       637.000000
max      1565.000000
Name: book, dtype: float64

In [470]:
book_list

array(['ME0053', 'ME0202', 'LP1096', 'LP1140', 'LB0067', 'LB0076',
       'LB0080', 'LB0083', 'LB0091', 'LB0097', 'LB0102', 'LB0161',
       'LB0282', 'LB0290', 'LB0350', 'LB0357', 'LB0362', 'LP0036',
       'LP0266', 'LP0982', 'LP1001', 'LP1156', 'MK0208', 'LB0107',
       'LB0122', 'LB0167', 'LB0236', 'LB0241', 'LB0253', 'LB0305',
       'LB0363', 'LP1036', 'LP1108', 'ME0213', 'ME0231', 'MK0095',
       'ME0221', 'ME0224', 'MK0031', 'MK0179', 'MK0205', 'LP0231',
       'ME0176', 'ME0262', 'MK0035', 'MK0094', 'MK0123', 'MK0130',
       'MK0150', 'MK0152', 'MK0225'], dtype=object)

In [473]:
sample_list = ['ME0202', 'MK0150', 'LP1096', 'LB0241']

In [477]:
sample_1 = new_df[new_df.book_id == sample_list[0]]
sample_2 = new_df[new_df.book_id == sample_list[1]]
sample_3 = new_df[new_df.book_id == sample_list[2]]
sample_4 = new_df[new_df.book_id == sample_list[3]]

In [512]:
def test_split(sample):
    import random
    return split_sent(sample.iloc[random.randint(0,len(sample)-1)].paragraphs[0])

In [541]:
sample = new_df[new_df['book_id'] == 'ME0053']

In [542]:
sample

,book_id,body_num,paragraphs
0,ME0053,1,"[서문 신경계 질환에 대한 물리치료 중재는 임상에서 중요한 영역을 차지하고 있으며,..."
1,ME0053,2,"[01 임상신경학 Clinical neurology, Keyword, 신경세포 (n..."
2,ME0053,3,[02 신경계 물리치료의 원리 Principle of neurological phy...
3,ME0053,4,"[03 뇌졸중 Stroke Keyword, 뇌졸중 (cerebral vascular..."
4,ME0053,5,"[04 파킨슨병 Parkinson’s diseases Keyword, 바닥핵 (ba..."
5,ME0053,6,"[05 치매 Dementia Keyword, 치매 (dementia) 인지 (cog..."
6,ME0053,7,"[06 소뇌 실조 Cerebellar ataxia, Keyword 실조 (ataxi..."
7,ME0053,8,"[07 뇌성마비 Cerebral palsy, Keyword 강직형 (spastic)..."
8,ME0053,9,"[08 외상성 뇌손상 Traumatic brain injury, Keyword 외상..."
9,ME0053,10,"[09 외상성 척수손상 Traumatic spinal cord injury, Key..."


In [574]:
new_df

,book_id,body_num,paragraphs
0,ME0053,1,"[서문 신경계 질환에 대한 물리치료 중재는 임상에서 중요한 영역을 차지하고 있으며,..."
1,ME0053,2,"[01 임상신경학 Clinical neurology, Keyword, 신경세포 (n..."
2,ME0053,3,[02 신경계 물리치료의 원리 Principle of neurological phy...
3,ME0053,4,"[03 뇌졸중 Stroke Keyword, 뇌졸중 (cerebral vascular..."
4,ME0053,5,"[04 파킨슨병 Parkinson’s diseases Keyword, 바닥핵 (ba..."
...,...,...,...
2,MK0225,3,[Chapter 3 재난과 질병 테러 및 바이러스 질환 1. 위험물질(hazardo...
3,MK0225,4,[Chapter 4 재난대책 한국의 재난관리대책 1. 재난관리체계의 역사적 전개와 ...
4,MK0225,5,[Chapter 5 재난응급의료체계 1. 병원 재난 계획의 원칙 1) 서론 이 장에...
5,MK0225,6,[Chapter 6 해외 긴급구조 의료장비 및 운영방안 서론 1. 개요 1) 재난 ...


In [577]:
sample.iloc[0].paragraphs

['서문 신경계 질환에 대한 물리치료 중재는 임상에서 중요한 영역을 차지하고 있으며, 물리치료를 공부하는 학생은 반드시 알고 있어야 하는 중요한 교과목이다. 최근의 물리치료사 국가고시 체계는 임상 실무 중심의 교육과정 이수를 강조하고 있으며, 물리치료 임상 실무에 대한 문제해결 중심으로 변화되고 있다. 이 책의 구성은 한국보건의료인 국가시험원 물리치료사 국가고시를 준비하기 위해 반드시 알아야 할 기본적인 내용과 최신의 내용들을 반영하였다. 이 책의 구성은 먼저 신경계의 구조와 기능, 각 구조의 손상에 따른 임상 양상을 이해하기 위해 임상신경학을 설명하였다. 또 신경계 질환 중재의 기본적인 생리학적 기전과 치료적 원리를 설명하여 전반적으로 이해할 수 있도록 하였다. 마지막으로 신경계 질환을 뇌 관련 질환, 척수 관련 질환, 말초신경 관련 질환, 유전자결함 및 면역매개 관련 질환으로 구분하였고, 각 질환에 따라 정의, 발생빈도, 원인 및 병인, 분류 및 구분, 임상 양상을 이해하고 질환자의 진단평가과정을 통한 물리치료와 의학적 처지 및 관리의 중재로 구성하였다. 그러므로 이 책은 물리치료를 공부하는 학생들에게 신경계 질환에 대한 전반적인 이해와 질환에 따른 임상 양상을 알기 쉽게 이해하고, 환자의 문제점 파악 및 문제 목록을 작성하는 데 도움이 될 것이다. 신경계 질환의 임상 양상에 따른 문제해결을 위한 중재를 적용할 수 있도록 교육하는데 목적을 두고 있다. 이러한 과정을 통해 학생들은 신경계 질환의 문제점을 파악할 수 있게 될 것이며, 물리치료 임상의사결정을 통해 평가와 중재를 적용할 수 있고 체계적으로 학습하는 데 도움이 될 것이다. “신경계질환 중재학 초판”이 출간되면서 물리치료과 학생들의 물리치료사 국가고시 대비를 위한 학습서로서 많은 도움이 되었으며, 임상 실무에서 신경계 질환의 이해와 신경계 질환의 임상 양상에 따른 문제를 해결하고 중재를 적용하는 데 도움이 되었다. 하지만 초판 집필 및 편집과정 중에 잘못 기술된 내용과 오류를 바로잡고, 최신의 객관성

In [572]:
split_sent(sample.iloc[0].paragraphs[0])

['서문 신경계 질환에 대한 물리치료 중재는 임상에서 중요한 영역을 차지하고 있으며, 물리치료를 공부하는 학생은 반드시 알고 있어야 하는 중요한 교과목이다.',
 '최근의 물리치료사 국가고시 체계는 임상 실무 중심의 교육과정 이수를 강조하고 있으며, 물리치료 임상 실무에 대한 문제해결 중심으로 변화되고 있다.',
 '이 책의 구성은 한국보건의료인 국가시험원 물리치료사 국가고시를 준비하기 위해 반드시 알아야 할 기본적인 내용과 최신의 내용들을 반영하였다.',
 '이 책의 구성은 먼저 신경계의 구조와 기능, 각 구조의 손상에 따른 임상 양상을 이해하기 위해 임상신경학을 설명하였다.',
 '또 신경계 질환 중재의 기본적인 생리학적 기전과 치료적 원리를 설명하여 전반적으로 이해할 수 있도록 하였다.',
 '마지막으로 신경계 질환을 뇌 관련 질환, 척수 관련 질환, 말초신경 관련 질환, 유전자결함 및 면역매개 관련 질환으로 구분하였고, 각 질환에 따라 정의, 발생빈도, 원인 및 병인, 분류 및 구분, 임상 양상을 이해하고 질환자의 진단평가과정을 통한 물리치료와 의학적 처지 및 관리의 중재로 구성하였다.',
 '그러므로 이 책은 물리치료를 공부하는 학생들에게 신경계 질환에 대한 전반적인 이해와 질환에 따른 임상 양상을 알기 쉽게 이해하고, 환자의 문제점 파악 및 문제 목록을 작성하는 데 도움이 될 것이다.',
 '신경계 질환의 임상 양상에 따른 문제해결을 위한 중재를 적용할 수 있도록 교육하는데 목적을 두고 있다.',
 '이러한 과정을 통해 학생들은 신경계 질환의 문제점을 파악할 수 있게 될 것이며, 물리치료 임상의사결정을 통해 평가와 중재를 적용할 수 있고 체계적으로 학습하는 데 도움이 될 것이다.',
 '“신경계질환 중재학 초판”이 출간되면서 물리치료과 학생들의 물리치료사 국가고시 대비를 위한 학습서로서 많은 도움이 되었으며, 임상 실무에서 신경계 질환의 이해와 신경계 질환의 임상 양상에 따른 문제를 해결하고 중재를 적용하는 데 도움이 되었다.',
 '하지만 초판 집

In [506]:
# 왔다(그림 6.1). 
test_split(sample_1)

['CHAPTER 06 보바스 접근법 Bobath Approach 1. 보바스 치료의 개념에 대하여 설명할 수 있다.',
 '2. 보바스 치료의 치료배경을 설명할 수 있다.',
 '3. 보바스 치료의 목표를 설명할 수 있다.',
 '4. 보바스 치료의 내용에 대해 설명할 수 있다.',
 '5. 보바스 치료의 치료방법을 수행할 수 있다.',
 '두드리기(tapping) 상반신경억제(reciprocal inhibition) 상반신경지배(reciprocal innervation) 신경발달학적 치료(neurodevelopmental therapy) 자세긴장도(postural tone) 촉진(facilitation) 핵심부위 조절(key point of control) 핸들링(handling) 1. 개요 1) 역사 보바스 접근법의 창시자는 보바스 부부로, 의사인 카렐 보바스(Karel Bobath)와 결혼한 물리치료사 Berta는 남편의 성을 받아 베르타 보바스(Berta Bobath)로 불렸고 이 부부를 임상이나 교재 등에서는 편리하게 “Bobath 부부”로 부른다.',
 '두 사람의 첫 만남은 1924년 베를린의 어느 한 체육관에서였으며 1941년 4월 결혼하여 부부가 되었다.',
 '의사와 물리치료사인 이 부부는 그들이 가지고 있는 장점을 활용하여 임상에서 서로 환자를 진료, 관찰하거나 치료하면서 얻은 다양한 지식과 경험을 정립하여 보바스 치료 이론을 만들어낸 것으로, 처음부터 완벽할 순 없지만 시간이 지나며 지속적으로 수정 보완되면서 발전되어 왔다(그림 6.1). Berta는 이완과 억제에 대한 임상적 연구를 실제적 관점에서 실시하였으며, 이를 통해 얻은 유용한 치료 정보들을 자신의 첫 논문인 “위운동신경세포 질환에 대한 새로운 치료법(a new treatment of lesions of the upper motor neuron)”을 통해 영국 물리치료협회지에 1948년에 발표하였고 이후 1954년에는 “중추신경계 환자에서 비정상적 자세반사 활동(abnormal

In [507]:
# 들 집락균에 의해 감염되기 쉬워진 다. 이와 같은 혼합감염은 -> 다. 와 같은 순번 거를 때의 예외로 처리되지 않음
test_split(sample_2)

['3-28 질병의 개요 혐기균(anaerobic bacteria)은 성장을 위해서 산소 가 적은 환경을 필요로 하는 세균으로 공기 중 10% 의 CO2가 존재하는 상태에서의 고형 배지 표면에서 는 성장할 수 없다.',
 '일반적으로 인체 감염과 관련되어 병원체로 작용하는 혐기균들(예를 들어, Bacteroides fragilis, Prevotella melaninogenica, Fusobacterium nucleatum)은 비교적 산소를 견디어(aerotolerant) 산소 존재 하에 대개 증식을 하지는 않지만 72시간 까지도 생존이 가능하다.',
 '혐기균은 사람과 동물의 점막 표면에 정상 균무리 로 존재하며 면역과 대사에 공생과 상호부조 역할을 한다.',
 '주요 존재 부위는 구강, 위장관, 피부, 여성 생 식기이다.',
 '구강에 있는 균무리는 주로 혐기균으로 구성되며 타액에는 109/mL, 치은에는 1012/mL에 이 른다.',
 '혐기균은 말단 회장 이전까지의 장에는 많이 존재하지 않는다.',
 '대장에서 전체 세균 수가 증가하는 것처럼 혐기균의 수도 매우 크게 증가한다.',
 '예를 들어, 대장에는 대변 1 g당 1011-1012의 미생물이 무 산소성 대 산소성 비율이 1000:1로 존재한다(표 3- 28-1).',
 '공생하는 혐기균은 숙주에서 생리적, 대사적, 면역학적 기능에 중요한 부분을 담당한다.',
 '중요한 역할중의 하나로 장의 정상 균무리로 존재하여 다른 세균의 집락을 막는다.',
 'Bacteroides와 다른 장내혐 기균은 탄수화물을 발효하여 휘발성 지방산을 생산 하여 에너지원으로 쓰인다.',
 '또한 장내 혐기균은 비 타민 K, 담즙산 생산에 도움을 준다.',
 '혐기균 감염은 숙주와 세균간의 조화로운 관계가 깨질 때 발생된다.',
 '점막 장벽이나 피부가 수술, 외상, 종양, 허혈, 괴사 등 국소 산소 환원 전위(oxidation- reduction potential)를 감소시키는 인자들에 의해 손 상 받았을 때 이들 집락균에 

In [578]:
sample_2.paragraphs.loc[49]

['3-28 질병의 개요 혐기균(anaerobic bacteria)은 성장을 위해서 산소 가 적은 환경을 필요로 하는 세균으로 공기 중 10% 의 CO2가 존재하는 상태에서의 고형 배지 표면에서 는 성장할 수 없다. 일반적으로 인체 감염과 관련되어 병원체로 작용하는 혐기균들(예를 들어, Bacteroides fragilis, Prevotella melaninogenica, Fusobacterium nucleatum)은 비교적 산소를 견디어(aerotolerant) 산소 존재 하에 대개 증식을 하지는 않지만 72시간 까지도 생존이 가능하다. 혐기균은 사람과 동물의 점막 표면에 정상 균무리 로 존재하며 면역과 대사에 공생과 상호부조 역할을 한다. 주요 존재 부위는 구강, 위장관, 피부, 여성 생 식기이다. 구강에 있는 균무리는 주로 혐기균으로 구성되며 타액에는 109/mL, 치은에는 1012/mL에 이 른다. 혐기균은 말단 회장 이전까지의 장에는 많이 존재하지 않는다. 대장에서 전체 세균 수가 증가하는 것처럼 혐기균의 수도 매우 크게 증가한다. 예를 들어, 대장에는 대변 1 g당 1011-1012의 미생물이 무 산소성 대 산소성 비율이 1000:1로 존재한다(표 3- 28-1). 공생하는 혐기균은 숙주에서 생리적, 대사적, 면역학적 기능에 중요한 부분을 담당한다. 중요한 역할중의 하나로 장의 정상 균무리로 존재하여 다른 세균의 집락을 막는다. Bacteroides와 다른 장내혐 기균은 탄수화물을 발효하여 휘발성 지방산을 생산 하여 에너지원으로 쓰인다. 또한 장내 혐기균은 비 타민 K, 담즙산 생산에 도움을 준다. 혐기균 감염은 숙주와 세균간의 조화로운 관계가 깨질 때 발생된다. 점막 장벽이나 피부가 수술, 외상, 종양, 허혈, 괴사 등 국소 산소 환원 전위(oxidation- reduction potential)를 감소시키는 인자들에 의해 손 상 받았을 때 이들 집락균에 의해 감염되기 쉬워진 다. 이와 같은 혼합감염은 두경부 감염(만성 부비동 염, 만성 중이염, 

In [523]:
split_sent(' '.join(sample_2.paragraphs.loc[49]))

['3-28 질병의 개요 혐기균(anaerobic bacteria)은 성장을 위해서 산소 가 적은 환경을 필요로 하는 세균으로 공기 중 10% 의 CO2가 존재하는 상태에서의 고형 배지 표면에서 는 성장할 수 없다.',
 '일반적으로 인체 감염과 관련되어 병원체로 작용하는 혐기균들(예를 들어, Bacteroides fragilis, Prevotella melaninogenica, Fusobacterium nucleatum)은 비교적 산소를 견디어(aerotolerant) 산소 존재 하에 대개 증식을 하지는 않지만 72시간 까지도 생존이 가능하다.',
 '혐기균은 사람과 동물의 점막 표면에 정상 균무리 로 존재하며 면역과 대사에 공생과 상호부조 역할을 한다.',
 '주요 존재 부위는 구강, 위장관, 피부, 여성 생 식기이다.',
 '구강에 있는 균무리는 주로 혐기균으로 구성되며 타액에는 109/mL, 치은에는 1012/mL에 이 른다.',
 '혐기균은 말단 회장 이전까지의 장에는 많이 존재하지 않는다.',
 '대장에서 전체 세균 수가 증가하는 것처럼 혐기균의 수도 매우 크게 증가한다.',
 '예를 들어, 대장에는 대변 1 g당 1011-1012의 미생물이 무 산소성 대 산소성 비율이 1000:1로 존재한다(표 3- 28-1).',
 '공생하는 혐기균은 숙주에서 생리적, 대사적, 면역학적 기능에 중요한 부분을 담당한다.',
 '중요한 역할중의 하나로 장의 정상 균무리로 존재하여 다른 세균의 집락을 막는다.',
 'Bacteroides와 다른 장내혐 기균은 탄수화물을 발효하여 휘발성 지방산을 생산 하여 에너지원으로 쓰인다.',
 '또한 장내 혐기균은 비 타민 K, 담즙산 생산에 도움을 준다.',
 '혐기균 감염은 숙주와 세균간의 조화로운 관계가 깨질 때 발생된다.',
 '점막 장벽이나 피부가 수술, 외상, 종양, 허혈, 괴사 등 국소 산소 환원 전위(oxidation- reduction potential)를 감소시키는 인자들에 의해 손 상 받았을 때 이들 집락균에 

In [511]:
# parsing 데이터 중 중간.(또는의 의미)를 마침표로 표기한 경우 잘 분리 안 됨
# 일정 글자 이하 + . 인 경우 예외로 추가
test_split(sample_3)

['동물보호법 제7장은 동물을 적정하게 보호 .',
 '관리하기 위하여 동물학대행위 등 이 법 위반 행위에 대한 벌칙, 양벌규정, 과태료를 규정하고 있다.',
 '동물을 합리적인 이유 없이 죽이거나, 잔인하게 죽이거나 동물에게 불필요 한 고통을 주는 등의 동물학대행위를 금지하고자 하는 목적은 동물보호법의 주 된 제정취지 중 하나이다.',
 '동물보호법은 1991. 5. 31. 제정 이래 여러 차례 개정 을 통해 동물학대행위 등 방지조항의 실효성을 높이기 위하여 관련 규정을 보완 하는 한편, 기존 제도의 운영상 나타난 일부 미비점을 개선 .',
 '보완해 왔다.',
 '주요 개정 내용을 요약하면 다음과 같다.',
 '∙ 2017. 3. 21. 일부개정: 동물학대행위자에 대해 ‘1년 이하의 징역 또는 1천만원 이하의 벌금’에 처하던 것을 ‘2년 이하의 징역 또는 2천만원 이 하의 벌금’에 처하도록 하여 벌칙을 상향하고, 이를 상습적으로 위반한 자를 가중처벌하는 조항을 마련하였다.',
 '또한, 도박을 목적으로 동물을 이용하거나 도박 .',
 '시합 .',
 '복권 .',
 '오락 .',
 '유흥 .',
 '광고 등의 상이나 경품으로 동 물을 제공한 자, 영리를 목적으로 동물을 대여한 자 등을 300만원 이하 의 벌금에 처하는 조항을 신설하였으며, 법인의 대표자, 개인의 대리인 등이 벌칙규정에 해당하는 처벌을 받는 경우 해당 법인이나 개인도 벌 금형으로 처벌받도록 하는 양벌규정을 신설하였다.',
 '∙ 2018. 3. 20. 일부개정: 맹견의 소유자가 관리의무를 위반하여 사람을 사망 혹은 상해를 입게 하는 경우 각각 3년 이하의 징역 또는 3천만원 이하의 벌금, 2년 이하의 징역 또는 2천만원 이하의 벌금으로 처벌할 수 있도록 하는 조항을 신설하고, 맹견에 대한 관리의무나 안전조치, 교육 이수 등을 하지 않은 소유자등에 대한 과태료 부과 조항을 신설하였다.',
 '∙ 2020. 2. 11. 일부개정: 동물을 죽음에 이르게 하는 학대행위를 한 자에 대한 처벌을 기존 

In [783]:
# 문장이 너무 긴 경우
split_sent(' '.join(sample_3.loc[7].paragraphs))

['동물보호법 제7장은 동물을 적정하게 보호 . 관리하기 위하여 동물학대행위 등 이 법 위반 행위에 대한 벌칙, 양벌규정, 과태료를 규정하고 있다.',
 '동물을 합리적인 이유 없이 죽이거나, 잔인하게 죽이거나 동물에게 불필요 한 고통을 주는 등의 동물학대행위를 금지하고자 하는 목적은 동물보호법의 주 된 제정취지 중 하나이다.',
 '동물보호법은 1991. 5. 31. 제정 이래 여러 차례 개정 을 통해 동물학대행위 등 방지조항의 실효성을 높이기 위하여 관련 규정을 보완 하는 한편, 기존 제도의 운영상 나타난 일부 미비점을 개선 . 보완해 왔다.',
 '주요 개정 내용을 요약하면 다음과 같다.',
 '∙ 2017. 3. 21. 일부개정: 동물학대행위자에 대해 ‘1년 이하의 징역 또는 1천만원 이하의 벌금’에 처하던 것을 ‘2년 이하의 징역 또는 2천만원 이 하의 벌금’에 처하도록 하여 벌칙을 상향하고, 이를 상습적으로 위반한 자를 가중처벌하는 조항을 마련하였다.',
 '또한, 도박을 목적으로 동물을 이용하거나 도박 . 시합 . 복권 . 오락 . 유흥 . 광고 등의 상이나 경품으로 동 물을 제공한 자, 영리를 목적으로 동물을 대여한 자 등을 300만원 이하 의 벌금에 처하는 조항을 신설하였으며, 법인의 대표자, 개인의 대리인 등이 벌칙규정에 해당하는 처벌을 받는 경우 해당 법인이나 개인도 벌 금형으로 처벌받도록 하는 양벌규정을 신설하였다.',
 '∙ 2018. 3. 20. 일부개정: 맹견의 소유자가 관리의무를 위반하여 사람을 사망 혹은 상해를 입게 하는 경우 각각 3년 이하의 징역 또는 3천만원 이하의 벌금, 2년 이하의 징역 또는 2천만원 이하의 벌금으로 처벌할 수 있도록 하는 조항을 신설하고, 맹견에 대한 관리의무나 안전조치, 교육 이수 등을 하지 않은 소유자등에 대한 과태료 부과 조항을 신설하였다.',
 '∙ 2020. 2. 11. 일부개정: 동물을 죽음에 이르게 하는 학대행위를 한 자에 대한 처벌을 기존 ‘2년 이하의 징역 또는 2천만원 이하의 벌금’에 

In [780]:
sample_3.loc[7].paragraphs

['동물보호법 제7장은 동물을 적정하게 보호 . 관리하기 위하여 동물학대행위 등 이 법 위반 행위에 대한 벌칙, 양벌규정, 과태료를 규정하고 있다. 동물을 합리적인 이유 없이 죽이거나, 잔인하게 죽이거나 동물에게 불필요 한 고통을 주는 등의 동물학대행위를 금지하고자 하는 목적은 동물보호법의 주 된 제정취지 중 하나이다. 동물보호법은 1991. 5. 31. 제정 이래 여러 차례 개정 을 통해 동물학대행위 등 방지조항의 실효성을 높이기 위하여 관련 규정을 보완 하는 한편, 기존 제도의 운영상 나타난 일부 미비점을 개선 . 보완해 왔다. 주요 개정 내용을 요약하면 다음과 같다. ∙ 2017. 3. 21. 일부개정: 동물학대행위자에 대해 ‘1년 이하의 징역 또는 1천만원 이하의 벌금’에 처하던 것을 ‘2년 이하의 징역 또는 2천만원 이 하의 벌금’에 처하도록 하여 벌칙을 상향하고, 이를 상습적으로 위반한 자를 가중처벌하는 조항을 마련하였다. 또한, 도박을 목적으로 동물을 이용하거나 도박 . 시합 . 복권 . 오락 . 유흥 . 광고 등의 상이나 경품으로 동 물을 제공한 자, 영리를 목적으로 동물을 대여한 자 등을 300만원 이하 의 벌금에 처하는 조항을 신설하였으며, 법인의 대표자, 개인의 대리인 등이 벌칙규정에 해당하는 처벌을 받는 경우 해당 법인이나 개인도 벌 금형으로 처벌받도록 하는 양벌규정을 신설하였다. ∙ 2018. 3. 20. 일부개정: 맹견의 소유자가 관리의무를 위반하여 사람을 사망 혹은 상해를 입게 하는 경우 각각 3년 이하의 징역 또는 3천만원 이하의 벌금, 2년 이하의 징역 또는 2천만원 이하의 벌금으로 처벌할 수 있도록 하는 조항을 신설하고, 맹견에 대한 관리의무나 안전조치, 교육 이수 등을 하지 않은 소유자등에 대한 과태료 부과 조항을 신설하였다. ∙ 2020. 2. 11. 일부개정: 동물을 죽음에 이르게 하는 학대행위를 한 자에 대한 처벌을 기존 ‘2년 이하의 징역 또는 2천만원 이하의 벌금’에 처하 던 것을 ‘3년 이하의 징역 또는 3천만원 이

In [525]:
test_split(sample_4)

['제 1 절 행정절차에 관한 기초적 이해 Ⅰ. 행정절차의 개념 행정절차란 넓은 의미로는 행정의사의 결정과 집행에 관련된 일체의 과정을 의미한다.',
 '여기에는 사전절차로서의 행정입법․행정행위․행정계획․공법계약에 관한 절차와 사후절차로서의 행정심판절차, 행정상 의무이행확보절차까지 포함된다.',
 '이에 대해 좁 은 의미의 행정절차란 제1차적 행정의사결정에 관한 사전절차, 즉 행정입법․행정행 위․행정계획․공법계약 등을 함에 있어서 밟아야 할 사전절차를 의미한다.',
 '후자가 가장 보편적인 행정절차개념으로 사용된다.',
 '행정절차법 역시 이에 맞춰 행정활동의 준비와 시작부터 결정 그 자체와 그 집행에 이르기까지의 행정의 의사형성의 일정한 방도와 형식을 정한다.',
 '행정절차법의 효시는 1948년에 제정된 미국 행정절차법(APA)이다.',
 '우리의 경우 오래 전부터 행정절차의 필요성은 널리 공감대가 형성되었으며, 도로법․하천법 등 개별법에서도 청문제도가 도입되었다.',
 '통일적인 행정절차법을 제정하기 위한 노력의 결과로서 1987년 행정절차법안이 입법예고까지 되었지만, 부처간의 이견과 미약한 주도력으로 좌초되었다.',
 '그 후 일본이 ‘행정수속법’을 1993년 11월에 제정하여 1994년 10월부터 시행한 것이 계기가 되어 1996년 12월 ‘행정절차법’이 제정되어 1998년부터 시행되어 왔다.',
 '현행법은 독일 행정절차법 및 1987년의 법안과 비교하여 오로지 절차 적 내용만을 목표로 삼았기에 아쉬움이 크다.',
 '행정법총칙에 해당하는 실체적 내용을 담아 새롭게 upgrade할 필요가 있다.1) 궁극적으로 공공법제를 21세기에 맞춰 현대화 1) 행정절차의 비교법적 특히 유럽법적 논의에 관해서는 김중권, 유럽화된 독일행정절차법에 관한 연구, 79면 이하.',
 '－ 547 － Chapter 행 / 정 / 법 05 행정절차법 548 제5 편 정보와 관련한 행정활동 한다는 차원에서 행정절차법을 행정활동 전반에 관한 규준을 담은 행정기본법의 제 정을 계

In [781]:
# 새로운 예외적 패턴 'vgl. ', 'S.80ff.', 'S. 2106ff.' "NJW 1976, S. 2106ff." "Wahl, DVBl. 2003, S.1285ff." (완료)
# .뒤에space가 없는 경우 처리되지 못하고 있음 
# 경우에는 그렇지 않다.28) 이건 각주 같은데 각주 때문에 처리 안 되는 경우

split_sent(' '.join(sample_4.loc[19].paragraphs))

['제 1 절 행정절차에 관한 기초적 이해 Ⅰ. 행정절차의 개념 행정절차란 넓은 의미로는 행정의사의 결정과 집행에 관련된 일체의 과정을 의미한다.',
 '여기에는 사전절차로서의 행정입법․행정행위․행정계획․공법계약에 관한 절차와 사후절차로서의 행정심판절차, 행정상 의무이행확보절차까지 포함된다.',
 '이에 대해 좁 은 의미의 행정절차란 제1차적 행정의사결정에 관한 사전절차, 즉 행정입법․행정행 위․행정계획․공법계약 등을 함에 있어서 밟아야 할 사전절차를 의미한다.',
 '후자가 가장 보편적인 행정절차개념으로 사용된다.',
 '행정절차법 역시 이에 맞춰 행정활동의 준비와 시작부터 결정 그 자체와 그 집행에 이르기까지의 행정의 의사형성의 일정한 방도와 형식을 정한다.',
 '행정절차법의 효시는 1948년에 제정된 미국 행정절차법(APA)이다.',
 '우리의 경우 오래 전부터 행정절차의 필요성은 널리 공감대가 형성되었으며, 도로법․하천법 등 개별법에서도 청문제도가 도입되었다.',
 '통일적인 행정절차법을 제정하기 위한 노력의 결과로서 1987년 행정절차법안이 입법예고까지 되었지만, 부처간의 이견과 미약한 주도력으로 좌초되었다.',
 '그 후 일본이 ‘행정수속법’을 1993년 11월에 제정하여 1994년 10월부터 시행한 것이 계기가 되어 1996년 12월 ‘행정절차법’이 제정되어 1998년부터 시행되어 왔다.',
 '현행법은 독일 행정절차법 및 1987년의 법안과 비교하여 오로지 절차 적 내용만을 목표로 삼았기에 아쉬움이 크다.',
 '행정법총칙에 해당하는 실체적 내용을 담아 새롭게 upgrade할 필요가 있다.1) 궁극적으로 공공법제를 21세기에 맞춰 현대화 1) 행정절차의 비교법적 특히 유럽법적 논의에 관해서는 김중권, 유럽화된 독일행정절차법에 관한 연구, 79면 이하.',
 '－ 547 － Chapter 행 / 정 / 법 05 행정절차법 548 제5 편 정보와 관련한 행정활동 한다는 차원에서 행정절차법을 행정활동 전반에 관한 규준을 담은 행정기본법의 제 정을 계

In [678]:
text = '63) Vgl. Dolde, NVwZ 2006, S.857ff. 594'
tokens = []

for sent in re.finditer('(\.\s)', text):
    start = sent.span()[0]
    end = sent.span()[1]
        
    if start < 6:
        token = text[start-start:end-1]
    else:
        token = text[start-6:end-1]
    tokens.append(token)



In [682]:
tokens

['3) Vgl.', '.857ff.']

In [760]:
re.findall('(S?\.?\s?\d*ff\.)', '1258ff.')

['1258ff.']

In [782]:
# 수정 후 

def split_sent(text:str)->list:
    # split 시 무시할 패턴 정의  
    # 숫자. 로 끝나는 경우 
    check = """(eqs?\.)|(figs?\.)|(sect\.)|\     # eq.(eqs.), fig.(figs.), sec.(SEC.),
    (sec\.)|(et\.)|(\s?al\.)|(\s?et\.?\sal\.)|\  #  et. al. 
    (S?\.?\s?\d*ff\.)$|(dvbl\.)|(vgl\.)|\        # 추가: '.857ff.','3) Vgl.'  11.18
    ^(e\.)|(\s?\(?[A-Z]{1,4}\.)|^([A-Z]\.)|\     # 시작 시 e. g. (S. S. 로 시작할 경우, ABCD. 와 같은 경우 추가(+11.18)
    ([가-힣]{1,2}\s\.)$|\                         # 다. 일경우만 제외하는 등 변경 필요
    (\(?e\.\s?g\.)|(\(?i\.\s?e\.)|\
    ([MDCLXVI]{1,2}\.)$\                         # 로마자 추가
    |(dix.)\
    |(no\.)|([0-9]\.)$|(vs\.)|(ref\.)|(cf\.)\    # no. 1. vs. ref.(Ref.) cf.(Cf.)
    """
    pp = re.compile(check, re.VERBOSE|re.IGNORECASE)
    na_count = 0

        
    # 예외처리: 문장 끝에 .(공백없음)으로 끝나거나 .이 없는 경우
    sample = new_sample(text)
    splited_list = []
    prev_end = 0
    
    try:
        # .으로 끝나는 모든 경우를 find
        for sent in re.finditer('(\.\s)', sample):
            start = sent.span()[0]
            end = sent.span()[1]

            if start < 6:
                token = sample[start-start:end-1]
            else:
                token = sample[start-6:end-1]

    #         print()
    #         print(pp.search(token))
    #         print(token)
    #         print(prev_end, end)
    #         print(len(sample))

            # '. ' 앞뒤에 check에 해당하는 패턴이 없는 경우 문장으로 나누기
            if (pp.search(token) == None):
                if re.search('(\s\(?\w\.\)?)$', token) == None:  # 위의 check에서 제대로 동작 안 해서 따로 집어 넣음
                    splited_list.append(sample[prev_end:end-1])
                    prev_end = end

            elif pp.search(token) != None:
                if len(sample) == end:
                    splited_list.append(sample[prev_end:end-1])
                    
    except:
        print(f'error occured!')
        print(sample)
    return splited_list

